In [85]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [86]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [87]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect,func

In [88]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///../Resources/hawaii.sqlite")

In [89]:
Base = automap_base()

In [90]:
# reflect an existing database into a new model

# reflect the tables
Base.prepare(autoload_with=engine)


In [91]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [92]:
# Save references to each table
Measurement = Base.classes.measurement

In [93]:
# Save references to each table
Station = Base.classes.station

In [94]:
# Create our session (link) from Python to the DB
session = Session(engine)


In [95]:
first_row_measurement = session.query(Measurement).first()
first_row_measurement.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7fa8116e6350>,
 'station': 'USC00519397',
 'date': '2010-01-01',
 'tobs': 18.3,
 'id': 0,
 'prcp': 2.0}

In [96]:
first_row_station = session.query(Station).first()
first_row_station.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7fa8112a17d0>,
 'station': 'USC00519397',
 'name': 'WAIKIKI 717.2, HI US',
 'longitude': -157.8168,
 'id': 0,
 'latitude': 21.2716,
 'elevation': 0.914}

# Exploratory Precipitation Analysis

In [97]:
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [98]:
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [99]:
# Total dates
session.query(func.count(Measurement.date)).all()

[(19550,)]

In [100]:
# Find the most recent date in the data set.
session.query(Measurement.date).order_by(Measurement.date).first()

('2010-01-01',)

In [101]:
# Find the most recent date in the data set.
session.query(Measurement.date).order_by(Measurement.date.desc()).first()

('2017-08-23',)

In [102]:
from datetime import datetime, timedelta

In [103]:
# sel = [Measurement.date, 
#       Measurement.prcp]
# year_2017 = session.query(*sel).\
#     filter(func.strftime("%y", Measurement.date) == "2017").all()
# year_2017

In [104]:
# date = dt.datetime(2017, 8, 23)
query_date = dt.date(2017, 8, 23) - dt.timedelta(days=362)
# months_12_from_last_date = (date) - dt.timedelta(months = 12)

results = session.query(Measurement.date, 
                        Measurement.prcp).\
                  filter(Measurement.date == query_date).all()

# List comprehension 
prcp_rows = [{"Date": result[0], "Precipitation": result[1]} for result in results]
prcp_rows

[{'Date': '2016-08-26', 'Precipitation': 0.0},
 {'Date': '2016-08-26', 'Precipitation': 0.8},
 {'Date': '2016-08-26', 'Precipitation': 0.5},
 {'Date': '2016-08-26', 'Precipitation': 1.0},
 {'Date': '2016-08-26', 'Precipitation': 0.0},
 {'Date': '2016-08-26', 'Precipitation': 0.3}]

In [105]:
# Close the session
session.close()

In [106]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.


# Perform a query to retrieve the data and precipitation scores


# Save the query results as a Pandas DataFrame and set the index to the date column


# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data




In [107]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [108]:
columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [109]:
# Design a query to calculate the total number stations in the dataset
session.query(func.count(Station.station)).all()


[(9,)]

In [110]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [111]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [112]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [113]:
# Close Session
session.close()